# LangChain - conceitos básicos

Este noteboook ilustra os conceitos básicos da biblioteca LangChain.

## Modelos de conversação (*chat Models*)

O conceito de **"chat model"** refere-se a modelos de linguagem que seguem o formato de **conversas estruturadas**.

> Um **chat model** é um modelo que recebe uma **sequência de mensagens** com papéis explícitos (`system`, `user`, `assistant`) e responde de forma contextualizada, como em um chat.

Um chat model recebe uma **lista de mensagens** como esta:

```python
  messages = [
      SystemMessage(content="You are a helpful assistant."),
      HumanMessage(content="What is the capital of Brazil?")
  ]
```

O modelo usa esse histórico para gerar uma resposta como:

```json
  AIMessage(content="The capital of Brazil is Brasília.")
```

Na LangChain, os chat models são instâncias de classes como:

  * `ChatOpenAI`
  * `ChatOllama`
  * `ChatAnthropic`

Eles seguem a interface da classe base `BaseChatModel`, e podem ser usados em cadeias (`Chains`), ferramentas (`Tools`) ou agentes (`Agents`).

Abaixo está uma hierarquia simplificada da classe `BaseChatModel`, que é a superclasse abstrata dos modelos de linguagem com interface de chat no LangChain:

```text
BaseChatModel
├── ChatOpenAI        → interface com os modelos da OpenAI (como gpt-3.5, gpt-4)
├── ChatAnthropic     → interface com Claude (Anthropic)
├── ChatOllama        → interface com modelos locais via Ollama (ex: gemma, llama3)
├── ChatGooglePalm    → interface com o PaLM da Google
├── ChatFireworks     → interface com modelos hospedados na FireworksAI
├── ChatCohere        → interface com o Cohere Chat
├── ChatAzureOpenAI   → variante da OpenAI para Azure
├── ... (outras integrações via LangChainHub ou LangChain Community)
```

In [5]:
%run ../get_llm.py

## Mensagens

Na LangChain, há várias classes para representar mensagens em uma conversa. Essas classes estruturam a conversa para modelos de chat (como `ChatOpenAI` ou `ChatOllama`) e são essenciais para manter **contexto, coerência e controle do comportamento** do agente. Os principais tipos de mensagens são os seguintes:

- `HumanMessage`: Representa uma **mensagem enviada pelo usuário**. É o que o modelo deve interpretar como entrada.

```python
    HumanMessage(content="What is the weather like today?")
```

- `AIMessage`: Representa uma **mensagem gerada pelo modelo (LLM)**. É usada para manter o histórico da conversa.

```python
    AIMessage(content="The weather today is sunny and warm.")
```

- `SystemMessage`: Fornece **instruções de configuração ou contexto** para o modelo. Define o comportamento esperado.

```python
    SystemMessage(content="You are a helpful assistant that answers concisely.")
```

No LangChain, a classe `ChatMessage` é a **superclasse** (ou classe base) de:

* `HumanMessage`
* `AIMessage`
* `SystemMessage`
* `FunctionMessage` (em alguns casos)
* `ToolMessage` (LangChain mais recente)

Hierarquia simplificada:

```text
ChatMessage
├── HumanMessage     → representa uma mensagem escrita por um humano
├── AIMessage        → representa a resposta do modelo (LLM)
├── SystemMessage    → configura o comportamento ou tom do modelo
├── FunctionMessage  → resultado da execução de uma função (caso use tools)
├── ToolMessage      → resposta de uma ferramenta externa (LangChain Tools)
```

---

Propósito:

* Permite que todas essas mensagens sejam tratadas de forma genérica como `ChatMessage`.
* Você pode percorrer uma lista de mensagens (`List[ChatMessage]`) sem se preocupar com o tipo específico — útil para histórico de conversas.
* Internamente, o modelo processa uma sequência de `ChatMessage`s com papéis diferentes.

Exemplo:

```python
    from langchain_core.messages import HumanMessage, AIMessage

    mensagens = [
        HumanMessage(content="Quem foi Marie Curie?"),
        AIMessage(content="Marie Curie foi uma cientista pioneira..."),
    ]

    for m in mensagens:
        print(f"{type(m).__name__}: {m.content}")
```

Saída:

```
HumanMessage: Quem foi Marie Curie?
AIMessage: Marie Curie foi uma cientista pioneira...
```

In [6]:
from langchain_core.messages import HumanMessage, AIMessage

mensagens = [
    HumanMessage(content="Quem foi Marie Curie?"),
    AIMessage(content="Marie Curie foi uma cientista pioneira..."),
]

for m in mensagens:
    print(f"{type(m).__name__}: {m.content}")

HumanMessage: Quem foi Marie Curie?
AIMessage: Marie Curie foi uma cientista pioneira...


## Interação com o modelo

Um método (função) importante da interface de `BaseChatModel` é `predict_messages`. Essa função é uma das diferentes formas (definidas na LangChain) para interagir com um LLM. Ela recebe uma sequência de mensagens (user, system, assistant) e retorna a próxima mensagem (gerada pelo modelo), mantendo o formato conversacional.

A sequência correta de mensagens na lista para chamar `predict_messages` deve sempre finalizar com `HumanMessage`, conforme a seguir:

```
[
    SystemMessage(...),       # configura o comportamento
    HumanMessage(...),        # pergunta 1
    AIMessage(...),           # resposta 1
    HumanMessage(...),        # pergunta 2 ← o modelo vai responder a essa
]
```

In [7]:
%run ../get_llm.py

#### Exemplo 1

In [8]:
from langchain.schema import HumanMessage

llm = get_llm(model_backend="ollama")
response = llm.predict_messages([
    HumanMessage(content="Tell me a joke about Artificial Intelligence.")
])
print(response.content)

/tmp/ipykernel_425047/875286630.py:4: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict_messages([


Why did the AI break up with the robot? 

... Because it said, "I need some space... and a little less processing power!" 😄 

---

Would you like to hear another joke?


#### Exemplo 2

No exemplo abaixo, a função `predict_messages` retorna um objeto `AIMessage` com o conteúdo gerado:

In [9]:
from langchain.schema import SystemMessage

response = llm.predict_messages([
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the capital of Brazil?")
])
print(response.content)

The capital of Brazil is **Brasília**. 

It’s a fascinating city – it was purpose-built as the capital in 1960! 😊 

Do you want to know anything more about Brasília or Brazil in general?


#### Exemplo 3

Neste exemplo:

- O modelo vê o contexto inteiro da conversa.

- Ele sabe que já falou sobre Marie Curie antes.

- A resposta à segunda pergunta pode ser mais precisa e direta.

In [10]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

llm = get_llm(model_backend="ollama")

# Histórico da conversa
messages = [
    SystemMessage(content="You are a helpful assistant that explains things clearly."),
    
    HumanMessage(content="Who is Marie Curie?"),
    AIMessage(content="Marie Curie was a pioneering physicist and chemist who conducted groundbreaking research on radioactivity. She was the first woman to win a Nobel Prize."),
    
    HumanMessage(content="What did she win the Nobel Prize for?")
]

# O LLM irá responder à última pergunta considerando o histórico
response = llm.predict_messages(messages)

# Imprime a resposta gerada
print(response.content)


Marie Curie actually won **two** Nobel Prizes! Here's a breakdown of what she won them for:

*   **1903 Nobel Prize in Physics:** This was jointly awarded to Marie, her husband Pierre Curie, and Henri Becquerel. They won for their research into **radioactivity**. Specifically, they investigated the properties of uranium rays, which Marie coined the term "radioactivity" to describe.

*   **1911 Nobel Prize in Chemistry:** This prize was awarded solely to Marie Curie for the discovery of the elements **polonium** and **radium**, and for isolating radium. This was a huge accomplishment – she was the first person to win Nobel Prizes in two different scientific fields.


It’s important to note that her work was incredibly difficult and dangerous, as she worked with radioactive materials without fully understanding the risks.

Do you want me to delve deeper into any specific aspect of her work or her life?


Diferenças entre chat model e LLM básico:

| Aspecto           | Chat Model                             | LLM Básico |
| ----------------- | -------------------------------------- | ----------------------- |
| Entrada           | Lista de mensagens (`role`, `content`) | Texto simples           |
| Formato de prompt | Estruturado                            | Prompt plano            |
| Exemplo típico    | ChatGPT, Claude, Gemini, Gemma         | Text-Davinci-003, GPT-J |
| Classe base       | `BaseChatModel`                        | `BaseLLM`               |

## Cadeias (Chains)

Em muitos casos simples, um chat model por si só é suficiente. Mas o conceito de `Chain` na LangChain vai além de apenas manter histórico: ele organiza a execução de etapas com reuso, modularidade e integração de múltiplos componentes.

Por que usar apenas um chat model pode ser suficiente?

Simplesmente isso:

```python
llm = get_llm()
messages = [...]
response = llm.predict_messages(messages)
```

resolve muitos casos — principalmente quando:

* Você **manualmente controla o histórico**
* Não há necessidade de **pré-processamento**, **pós-processamento**, ou **acesso a ferramentas**

---

Então, por que usar uma `Chain`? Porque em aplicações reais, geralmente precisamos estruturar algo como:

> Entrada → preparar prompt → gerar resposta → extrair algo → formatar saída → logar/armazenar

E isso pode envolver:

* Integração com **retrievers**
* Aplicação de **memória automática**
* Uso de **ferramentas externas**
* Extração de **informações estruturadas**
* Pós-processamento da saída do LLM

Comparando:

| Situação                                   | Chat Model direto | Chain recomendada? |
| ------------------------------------------ | ----------------- | ------------------ |
| Simples conversa em linguagem natural      |  Sim             |  Não precisa      |
| Montagem dinâmica de prompt                |  Manual         |  Sim              |
| Consulta a base de conhecimento (RAG)      |  Não             |  Sim              |
| Múltiplos passos (ex: busca + sumarização) |  Não             |  Sim              |
| Logging, callbacks, parsing estruturado    |  Não             |  Sim              |

Analogia didática:

> Usar um chat model direto é como usar uma **calculadora científica**: útil e pontual.
> Usar uma `Chain` é como **programar uma planilha com várias etapas**: você automatiza o processo, pode reutilizar partes, e conectar com outras fontes de dados.

#### Exemplo 1

In [11]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("Translate to French: {text}")
llm = get_llm(model_backend="ollama")

# Composição de operadores
chain = prompt | llm

output = chain.invoke({"text": "I love apples."})
print(output.content)

The most common and natural translation of "I love apples" in French is:

**J'aime les pommes.**

Here's a breakdown:

*   **J'aime** - I love
*   **les** - the (plural)
*   **pommes** - apples

You could also say:

*   **J'adore les pommes.** (This is a stronger expression of love - "I adore apples")


#### Exemplo 2

In [12]:
from langchain_core.prompts import PromptTemplate

# 1. Cadeia de resumo
summary_prompt = PromptTemplate.from_template("Summarize the following text:\n\n{text}")
summary_chain = summary_prompt | get_llm(model_backend="ollama")

# 2. Cadeia de tradução
translate_prompt = PromptTemplate.from_template("Translate to Portuguese:\n\n{text}")
translate_chain = translate_prompt | get_llm(model_backend="ollama")

# 3. Cadeia composta
def full_chain(article_text):
    summary = summary_chain.invoke({"text": article_text})
    translated = translate_chain.invoke({"text": summary})
    return {"summary": summary, "translated": translated}


In [13]:
# %load get_llm.py
def get_llm(model_backend, model_name=None):
    if model_backend == "openai":
        from langchain.chat_models import ChatOpenAI
        return ChatOpenAI(
            temperature=0,
            model=model_name or "gpt-3.5-turbo"
        )
    elif model_backend == "ollama":
        from langchain_ollama import ChatOllama
        return ChatOllama(
            temperature=0,
            model=model_name or "gemma3:latest"
        )
    else:
        raise ValueError(f"Unknown backend: {model_backend}")


In [14]:
# Choose between "openai" or "ollama"
# MODEL_BACKEND = "openai"
MODEL_BACKEND = "ollama"

In [15]:
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain.tools import DuckDuckGoSearchRun
from langchain.chains.llm_math.base import LLMMathChain
from langchain.utilities import SerpAPIWrapper

# Initialize the language model
llm = get_llm(MODEL_BACKEND)

Podemos inspecionar o prompt definido internamente na classe LLMMAthChain:

>     agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION

| Parte         | Significado                                                                                                                 |
| ------------- | --------------------------------------------------------------------------------------------------------------------------- |
| `CHAT`        | Usa um modelo de chat (ex: `ChatOpenAI`, `ChatAnthropic`) que aceita mensagens estruturadas (`System`, `User`, `Assistant`) |
| `ZERO_SHOT`   | Não requer exemplos (few-shot) para instruir o modelo — apenas uma descrição textual da tarefa e das ferramentas            |
| `REACT`       | O agente segue o ciclo: **Thought → Action → Observation**                                                                  |
| `DESCRIPTION` | As ferramentas são descritas por texto, e o modelo escolhe qual usar com base nessas descrições                             |
